Syntax natürlicher Sprachen, WS 2023/24

# 07 - Übung

In [1]:
import nltk
from nltk import Tree
from nltk import DependencyGraph
from spacy import displacy
from itertools import chain

In [2]:
def _tree_labeled(self, i):
    node = self.get_by_address(i)
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word + '(' + rel + ')', [self._tree_labeled(dep) for dep in deps]) if deps else word + '(' + rel + ')'

def tree_labeled(self):
    node = self.root
    word, rel = node["word"], node["rel"]
    deps = sorted(chain.from_iterable(node["deps"].values()))
    return Tree(word+'('+rel+')', [self._tree_labeled(dep) for dep in deps])

DependencyGraph._tree_labeled = _tree_labeled
DependencyGraph.tree_labeled = tree_labeled

def transform_nr_conll(sent_nr):
    sent_list = []
    for line in list(filter(None, sent_nr.split("\n"))):
        line_list = line.split(); line_list.pop(0); line_list.insert(1,"_")
        sent_list.append(" ".join([i for i in line_list[0:]]))
    return "\n".join([i for i in sent_list[0:]])

def displacy_dep_input(sent):
    deps = []
    for dep in sent.split('\n'):
        deps.append(dep.split())
    deps = [x for x in deps if x]
    ex, word_list, arc_list = [], [], []
    for index, dep in enumerate(deps):
        word_list.append({"text": dep[0], "tag": ""})
        line = index+1; head = int(dep[2]); label = dep[3]
        if head>line:
            start = index; end = head-1; direction = "left"
        else:
            start = head-1; end = index; direction = "right"
        if(label.lower() != "root"):
            arc_list.append({"start": start, "end": end, "label": label, "dir": direction})
    ex.append({"words": word_list,"arcs": arc_list})
    return ex

def show_dep_trees(sent_nr,style=1):
    sent = transform_nr_conll(sent_nr)
    dg = DependencyGraph(sent)
    if style == 0 or style == 2:
        tree_labeled = dg.tree_labeled()
        tree_labeled.pretty_print(unicodelines=True)   
    if style == 1 or style == 2:
        ex = displacy_dep_input(sent)
        html = displacy.render(ex, style="dep", manual=True, options={'distance':100})

## Aufgabe 1 - Analyse komplexer Satzkonstruktionen

### a) Schreiben Sie eine CFG, so dass für den folgende ambigen Satz zwei linguistisch korrekte Syntaxanalysen ausgegeben werden. Geben Sie auch die entsprechenden Dependenzanalysen für die beiden Strukturanalysen an.

- *Haustiere denken aber wilde Tiere jagen*



In [3]:
sent = 'Haustiere denken aber wilde Tiere jagen'

#### Konstituentenanalyse:

In [ ]:
grammar = nltk.CFG.fromstring("""
    
    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'denken' | 'jagen'
    CC -> 'aber'
    COMP -> 
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

#### Dependenzanalyse:

In [ ]:
# Analyse 1
sent_nr = """
1 Haustiere 
2 denken 
3 aber 
4 wilde 
5 Tiere 
6 jagen 
"""
show_dep_trees(sent_nr)

In [ ]:
# Analyse 2
sent_nr = """
1 Haustiere 
2 denken 
3 aber 
4 wilde 
5 Tiere 
6 jagen 
"""
show_dep_trees(sent_nr)

### b) Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Haustiere hoffen wilde Tiere zu jagen*


In [4]:
sent = 'Haustiere hoffen wilde Tiere zu jagen'

#### Konstituentenanalyse:

In [ ]:
grammar = nltk.CFG.fromstring("""
    
    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'hoffen' | 'jagen'
    COMP -> 'zu'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

#### Dependenzanalyse:

In [ ]:
sent_nr = """
1 Haustiere 
2 hoffen 
3 wilde 
4 Tiere 
5 zu 
6 jagen 
"""
show_dep_trees(sent_nr)

### c) Schreiben Sie eine CFG, so dass für den folgende Satz eine korrekte Syntaxanalyse ausgegeben wird. Geben Sie auch eine entsprechende Dependenzanalyse an.

- *Haustiere denken wilde Tiere zu jagen ist falsch*


In [5]:
sent = 'Haustiere denken wilde Tiere zu jagen ist falsch'

#### Konstituentenanalyse:

In [ ]:
grammar = nltk.CFG.fromstring("""
    
    N -> 'Haustiere' | 'Tiere'
    ADJ -> 'wilde'
    V -> 'denken' | 'jagen'
    COMP -> 'zu' 
    COP -> 'ist'
    ADJ -> 'falsch'
""")

parser = nltk.ChartParser(grammar)
trees = list(parser.parse(sent.split()))
if trees: [tree.pretty_print(unicodelines=True) for tree in trees]
else: print(f"no parse found for: {sent}")

#### Dependenzanalyse:

In [ ]:
sent_nr = """
1 Haustiere 
2 denken 
3 wilde 
4 Tiere 
5 zu 
6 jagen 
7 ist 
8 falsch 
"""
show_dep_trees(sent_nr)

## Aufgabe 2: Satzklassifikation und Syntaxregeln komplexer Sätze

#### Geben Sie (1.) zu der jeweils auftretenden komplexen Satzkonstruktion den Satz- bzw. Konstruktionstyp an. 

#### Geben Sie (2.) zu den CFG-Outputs des Stanford-Parsers (trainiert auf Penn-Treebank) anhand der Ableitungsbäume die Regeln der komplexen Satzkonstruktionen an.

#### Analysieren Sie (3.) anschließend die Dependenzstruktur der Sätze im UD-Schema.


- Penn-Treebank Tagset: http://surdeanu.cs.arizona.edu//mihai/teaching/ista555-fall13/readings/PennTreebankConstituents.html#Clause
- UD Dependency Tagset: https://universaldependencies.org/u/dep/all.html

---
### a)  *I believe that you win.*

In [6]:
sentence = "I believe that you win."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

### 2. Konstituentenanalyse:

In [7]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP believe) (SBAR (IN that) (S (NP (PRP you)) (VP (VBP win))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                     
             │                        
             S                       
 ┌───────────┴────┬────────────────┐  
 │                VP               │ 
 │     ┌──────────┴───┐            │  
 │     │             SBAR          │ 
 │     │     ┌────────┴────┐       │  
 │     │     │             S       │ 
 │     │     │        ┌────┴───┐   │  
 NP    │     │        NP       VP  │ 
 │     │     │        │        │   │  
PRP   VBP    IN      PRP      VBP  . 
 │     │     │        │        │   │  
 I  believe that     you      win  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 I 
2 believe 
3 that 
4 you 
5 win 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### b) *I want to believe.*

In [8]:
sentence = "I want to believe."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

#### Zusatzfrage: Welche Form der Kontrolle liegt hier vor? (Subjekt- oder Objektkontrolle?)

In [ ]:
#

#### Welche Form der Kontrolle liegt in folgendem Satz vor: *Sie überreden ihn, nach München zu fahren?*


In [ ]:
#

#### Welche satzwertigen Funktionen können Infinitivkonstruktionen erfüllen? 

In [ ]:
#

### 2. Konstituentenanalyse:

In [9]:
tree_str = '(ROOT (S (NP (PRP I)) (VP (VBP want) (S (VP (TO to) (VP (VB believe))))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                
          │                   
          S                  
 ┌────────┼────────────────┐  
 │        VP               │ 
 │   ┌────┴────┐           │  
 │   │         S           │ 
 │   │         │           │  
 │   │         VP          │ 
 │   │    ┌────┴─────┐     │  
 NP  │    │          VP    │ 
 │   │    │          │     │  
PRP VBP   TO         VB    . 
 │   │    │          │     │  
 I  want  to      believe  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 I 
2 want 
3 to 
4 believe 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### c) *the man who knew too much*

In [10]:
sentence = "the man who knew too much"

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

#### Zusatzfrage: Welche Attributsatztypen haben Sie in der Vorlesung kennengelernt? 

In [ ]:
#

### 2. Konstituentenanalyse:

In [11]:
tree_str = '(ROOT (FRAG (NP (NP (DT the) (NN man)) (SBAR (WHNP (WP who)) (S (VP (VBD knew) (ADJP (RB too) (JJ much))))))))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

            ROOT                        
             │                           
            FRAG                        
             │                           
             NP                         
     ┌───────┴─────────┐                 
     │                SBAR              
     │       ┌─────────┴────┐            
     │       │              S           
     │       │              │            
     │       │              VP          
     │       │    ┌─────────┴───┐        
     NP     WHNP  │            ADJP     
 ┌───┴───┐   │    │         ┌───┴────┐   
 DT      NN  WP  VBD        RB       JJ 
 │       │   │    │         │        │   
the     man who  knew      too      much



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 the 
2 man 
3 who 
4 knew 
5 too 
6 much 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#


### d) *They left, but they came back.* 

In [12]:
sentence = "They left, but they came back."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

### 2. Konstituentenanalyse:

In [13]:
tree_str = '(ROOT (S (S (NP (PRP They)) (VP (VBD left))) (, ,) (CC but) (S (NP (PRP they)) (VP (VBD came) (ADVP (RB back)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                       
                   │                          
                   S                         
      ┌────────┬───┼─────────┬─────────────┐  
      │        │   │         S             │ 
      │        │   │    ┌────┴────┐        │  
      S        │   │    │         VP       │ 
 ┌────┴───┐    │   │    │    ┌────┴───┐    │  
 NP       VP   │   │    NP   │       ADVP  │ 
 │        │    │   │    │    │        │    │  
PRP      VBD   ,   CC  PRP  VBD       RB   . 
 │        │    │   │    │    │        │    │  
They     left  ,  but  they came     back  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 they 
2 left 
3 but 
4 they 
5 came 
6 back 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### e) *This can't be right.*

In [14]:
sentence = "This can't be right."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

### 2. Konstituentenanalyse:

In [15]:
tree_str = "(ROOT (S (NP (DT This)) (VP (MD ca) (RB n't) (VP (VB be) (ADJP (JJ right)))) (. .)))"
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

         ROOT                  
          │                     
          S                    
 ┌────────┼──────────────────┐  
 │        VP                 │ 
 │    ┌───┼────────┐         │  
 │    │   │        VP        │ 
 │    │   │    ┌───┴────┐    │  
 NP   │   │    │       ADJP  │ 
 │    │   │    │        │    │  
 DT   MD  RB   VB       JJ   . 
 │    │   │    │        │    │  
This  ca n't   be     right  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

HINWEIS Negation:
- `advmod`, vgl. https://universaldependencies.org/de/dep/advmod.html
- oder sprachspezifischer zusätzlicher `neg`-Marker: https://universaldependencies.org/docs/u/dep/neg.html

In [ ]:
sent_nr = """
1 this 
2 ca 
3 n\'t 
4 be 
5 right 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#


### f) *When the storm started, she was still outside.*

In [16]:
sentence = "When the storm started, she was still outside."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

### 2. Konstituentenanalyse:

In [17]:
tree_str = '(ROOT (S (SBAR (WHADVP (WRB When)) (S (NP (DT the) (NN storm)) (VP (VBD started)))) (, ,) (NP (PRP she)) (VP (VBD was) (ADVP (RB still)) (ADVP (RB outside))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                              ROOT                          
                               │                             
                               S                            
            ┌──────────────────┼────┬────────┬────────────┐  
           SBAR                │    │        │            │ 
  ┌─────────┴─────┐            │    │        │            │  
  │               S            │    │        VP           │ 
  │         ┌─────┴──────┐     │    │   ┌────┼──────┐     │  
WHADVP      NP           VP    │    NP  │   ADVP   ADVP   │ 
  │     ┌───┴─────┐      │     │    │   │    │      │     │  
 WRB    DT        NN    VBD    ,   PRP VBD   RB     RB    . 
  │     │         │      │     │    │   │    │      │     │  
 When  the      storm started  ,   she was still outside  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 when 
2 the 
3 storm 
4 started 
5 she 
6 was 
7 still 
8 outside  
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#


### g) *To go now would be a mistake.* 

In [18]:
sentence = "To go now would be a mistake."

### 1. Fragen:

#### Was für eine Satzkonstruktion liegt hier vor? (koordiniert?, subordiniert?, keines von beiden?)


In [ ]:
#

#### Welcher eingebettete Satztyp liegt vor? (Subjekt-, Objekt-, Kausal-, Lokal-, Modal-, Temporal-, Relativsatz?)


In [ ]:
#

#### Wie läßt sich dieser Satztyp klassifizieren? (Komplement-, Adverbial-, Attributsatz?)


In [ ]:
#

### 2. Konstituentenanalyse:

In [19]:
tree_str = '(ROOT (S (S (VP (TO To) (VP (VB go) (ADVP (RB now))))) (VP (MD would) (VP (VB be) (NP (DT a) (NN mistake)))) (. .)))'
Tree.fromstring(tree_str).pretty_print(unicodelines=True)

                  ROOT                            
                   │                               
                   S                              
     ┌─────────────┴────────┬───────────────────┐  
     S                      │                   │ 
     │                      │                   │  
     VP                     VP                  │ 
 ┌───┴───┐         ┌────────┴───┐               │  
 │       VP        │            VP              │ 
 │   ┌───┴───┐     │    ┌───────┴───┐           │  
 │   │      ADVP   │    │           NP          │ 
 │   │       │     │    │       ┌───┴─────┐     │  
 TO  VB      RB    MD   VB      DT        NN    . 
 │   │       │     │    │       │         │     │  
 To  go     now  would  be      a      mistake  . 



#### Geben Sie die CFG-Ableitungsregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

### 3. Dependenzanalyse:

In [ ]:
sent_nr = """
1 to 
2 go 
3 now 
4 would 
5 be 
6 a 
7 mistake 
"""

show_dep_trees(sent_nr, style=2)

#### Geben Sie die Dependenzregeln für diese komplexe Satzkonstruktion an:

In [ ]:
#
#

## *Aufgabe 3 - Attributtypen und Attributfunktionen*

#### Analysieren Sie die folgende komplexe NP. Welche Attributtypen und -funktionen treten auf?
- *Die lang ersehnte Fahrt der Klasse 5b nach Paris, die sich aus irgendeinem Grunde immer wieder verzögert hatte, findet nun endlich statt.*

In [ ]:
#

In [20]:
sent_nr = """
1 Die 4 det
2 lang 3 advmod
3 ersehnte 4 acl
4 Fahrt 19 nsubj
5 der 6 det
6 Klasse 4 nmod 
7 5b 6 appos
8 nach 9 case
9 Paris 6 nmod 
10 die 17 nsbuj
11 sich 17 expl:pv
12 aus 14 case
13 irgendeinem 14 det
14 Grunde 17 obl
15 immer 16 advmod
16 wieder 17 advmod 
17 verzögert 4 acl:relcl
18 hatte 17 aux 
19 findet 0 ROOT
20 nun 19 advmod
21 endlich 19 advmod
22 statt 19 compound:prt
"""
show_dep_trees(sent_nr, style=0)

                                                                            findet(ROOT)                                                                                                            
     ┌─────────────┬───────────────┬─────────────────────────────────────────────┴────────────────────────┐                                                                                          
     │             │               │                                                                 Fahrt(nsubj)                                                                                   
     │             │               │           ┌───────────┬─────────────────────┬────────────────────────┴───────────────────────────┐                                                              
     │             │               │           │           │                Klasse(nmod)                                        verzögert(acl:                                                      
     │       

## *Aufgabe 4 - Wortstellung im Nebensatz*

#### Betrachten Sie die folgenden beiden Sätze:
- *Peter sagt, er habe kein Geld.*
- *Peter sagt, dass er kein Geld habe.*

#### Der erste Nebensatz  ist durch eine Konjunktion eingeleitet, der zweite Nebensatz ist ohne Einleitewort. Leiten Sie daraus eine Regularität zur Stellung des finiten Verbs im Nebensatz ab.

### a) Wenn der Nebensatz durch eine Konjunktion eingeleitet wird, wo steht diese im Stellungsfeldermodell?

In [ ]:
# 

### b) Von welchem Satzteil wird diese Position gefüllt, wenn die Konjunktion fehlt?

In [ ]:
# 

### c) Kann diese Position theoretisch auch leer bleiben?

In [44]:
# 